In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
def SelectFeature(feature_candidates,features_used,targets,df):
    N = len(feature_candidates)
    R2 = np.zeros(N)
    for k in range(N):   
        features_current = features_used.copy()
        features_current.append(feature_candidates[k])
        features = df[features_current]
        results = cross_validate(lr,features,targets,n_jobs=-1)
        R2[k] = results['test_score'].mean()
        
    R2_max = R2.max()
    feature_selected = feature_candidates[R2.argmax()] 
    return (feature_selected,R2_max)

# Use cross-validation, forward feature selection and/or feature engineering to predict the compressive strength of concrete using the features contained in the file *Concrete_train.csv* in the following ways

## (A)
### Develop a regressor that stresses interpretability

In [5]:
df = pd.read_csv('./Concrete_train.csv')
df.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,233.81,0.0,94.58,197.89,4.567,947.04,852.16,28,22.835445
1,251.81,0.0,99.94,146.14,12.350,1006.00,899.76,56,44.140254
2,252.00,0.0,0.00,186.00,0.000,1111.00,784.00,7,11.465986
3,325.60,166.4,0.00,174.00,8.900,881.60,790.00,28,61.235811
4,250.00,0.0,95.69,191.84,5.330,948.90,857.20,14,24.655662


In [6]:
features = df.drop(df.columns[-1],axis=1)
targets  = df.iloc[:,-1]
features = (features - features.mean())/features.std()

In [7]:
lr = LinearRegression()
feature_candidates = list(features.columns)
features_used = []
R2 = []

while (len(feature_candidates) > 0):
    print('candidates left:',len(feature_candidates),end='')
    (feature_selected,R2_max) = SelectFeature(feature_candidates,features_used,targets,features)
    features_used.append(feature_selected)
    feature_candidates.remove(feature_selected)
    R2.append(R2_max)
    print('  selected:',feature_selected)
    
results = pd.DataFrame()
results['features'] = features_used
results['test R-squared'] = R2

print()
print(results)
ix = results['test R-squared'].idxmax()
print()
print('use features 0 to',ix,end='')
R2_max = results['test R-squared'].max()
print('    maximum test R-squared =',R2_max.round(3))

candidates left: 8

C:\Users\micha\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


  selected: Cement (component 1)(kg in a m^3 mixture)
candidates left: 7  selected: Superplasticizer (component 5)(kg in a m^3 mixture)
candidates left: 6  selected: Age (day)
candidates left: 5  selected: Blast Furnace Slag (component 2)(kg in a m^3 mixture)
candidates left: 4  selected: Water  (component 4)(kg in a m^3 mixture)
candidates left: 3  selected: Fly Ash (component 3)(kg in a m^3 mixture)
candidates left: 2  selected: Coarse Aggregate  (component 6)(kg in a m^3 mixture)
candidates left: 1  selected: Fine Aggregate (component 7)(kg in a m^3 mixture)

                                            features  test R-squared
0          Cement (component 1)(kg in a m^3 mixture)        0.243398
1  Superplasticizer (component 5)(kg in a m^3 mix...        0.358201
2                                          Age (day)        0.477806
3  Blast Furnace Slag (component 2)(kg in a m^3 m...        0.547278
4          Water  (component 4)(kg in a m^3 mixture)        0.592691
5         Fly Ash

In [8]:
lr.fit(features[features_used],targets)
coef = pd.Series(lr.coef_,index=features.columns)
coef.sort_values(ascending=False)

Cement (component 1)(kg in a m^3 mixture)                13.159616
Water  (component 4)(kg in a m^3 mixture)                 9.806000
Fly Ash (component 3)(kg in a m^3 mixture)                7.056381
Coarse Aggregate  (component 6)(kg in a m^3 mixture)      6.446821
Age (day)                                                 2.136015
Fine Aggregate (component 7)(kg in a m^3 mixture)         2.117328
Blast Furnace Slag (component 2)(kg in a m^3 mixture)     1.644456
Superplasticizer (component 5)(kg in a m^3 mixture)      -3.031667
dtype: float64

#### We must drop the *Fine Aggregate* feature. We care about the amount of cement. We can see that the Amount of *Superplasticizer* has a negative effect on concrete strength (-3.031667).

## (B)
### Develop a regressor that stresses prediction accuracy.

In [9]:
features = df.drop(df.columns[-1],axis=1)
targets  = df.iloc[:,-1]
poly = PolynomialFeatures(2)
features_engineered = poly.fit_transform(features)
cols = poly.get_feature_names(features.columns)
features_engineered = pd.DataFrame(features_engineered,columns=cols)
features_engineered.head(3)
print('features shape =',features.shape)
print('features (engineered) shape =',features_engineered.shape)

features shape = (700, 8)
features (engineered) shape = (700, 45)


In [10]:
ix = (features_engineered.std() == 0)
drop_cols = features_engineered.columns[ix]
features_engineered = features_engineered.drop(drop_cols,axis=1)
print('features (engineered) shape =',features_engineered.shape)

features (engineered) shape = (700, 44)


In [11]:
features_engineered = (features_engineered - features_engineered.mean())/features_engineered.std()
features_engineered.describe()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),Cement (component 1)(kg in a m^3 mixture)^2,Cement (component 1)(kg in a m^3 mixture) Blast Furnace Slag (component 2)(kg in a m^3 mixture),...,Superplasticizer (component 5)(kg in a m^3 mixture)^2,Superplasticizer (component 5)(kg in a m^3 mixture) Coarse Aggregate (component 6)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture) Fine Aggregate (component 7)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture) Age (day),Coarse Aggregate (component 6)(kg in a m^3 mixture)^2,Coarse Aggregate (component 6)(kg in a m^3 mixture) Fine Aggregate (component 7)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture) Age (day),Fine Aggregate (component 7)(kg in a m^3 mixture)^2,Fine Aggregate (component 7)(kg in a m^3 mixture) Age (day),Age (day)^2
count,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,...,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02,7.000000e+02
mean,-4.726378e-16,8.754902e-17,-4.004733e-16,4.365793e-15,1.297375e-15,2.245410e-14,2.258194e-15,1.007131e-17,5.900042e-17,-5.129230e-16,...,2.127663e-16,-9.443240e-16,-9.714451e-17,-1.583257e-16,-9.766156e-15,3.610921e-15,-4.279910e-16,-1.035997e-15,-1.573345e-16,6.288620e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.705731e+00,-8.519567e-01,-8.645892e-01,-2.753839e+00,-1.065575e+00,-2.179938e+00,-2.196943e+00,-6.991254e-01,-1.204853e+00,-8.007647e-01,...,-6.282452e-01,-1.088724e+00,-1.023600e+00,-6.789082e-01,-2.027543e+00,-2.576520e+00,-7.047443e-01,-2.016521e+00,-7.309118e-01,-3.083660e-01
25%,-8.508288e-01,-8.519567e-01,-8.645892e-01,-7.748583e-01,-1.065575e+00,-5.043832e-01,-5.498518e-01,-5.223177e-01,-8.045954e-01,-8.007647e-01,...,-6.282452e-01,-1.088724e+00,-1.023600e+00,-6.789082e-01,-5.339743e-01,-6.733841e-01,-5.134557e-01,-5.930749e-01,-5.315415e-01,-2.987839e-01
50%,-1.545651e-01,-6.182263e-01,-8.645892e-01,1.469845e-01,8.660600e-02,-5.569256e-02,6.954236e-02,-2.972896e-01,-2.994707e-01,-4.859822e-01,...,-2.374297e-01,8.583567e-02,2.450493e-02,-3.516462e-01,-9.568767e-02,6.124728e-02,-3.085203e-01,1.721510e-02,-3.044342e-01,-2.686542e-01
75%,6.777092e-01,8.399590e-01,9.587934e-01,5.118232e-01,6.643760e-01,7.318183e-01,6.343291e-01,1.527665e-01,5.151788e-01,5.489637e-01,...,2.527996e-01,6.774174e-01,5.936573e-01,2.423668e-01,7.127135e-01,7.984935e-01,1.306575e-01,6.094322e-01,1.794333e-01,-1.481356e-01
max,2.520774e+00,3.348178e+00,2.217792e+00,2.990480e+00,4.342621e+00,2.083134e+00,2.677981e+00,5.119457e+00,3.136970e+00,3.258547e+00,...,7.982392e+00,3.853199e+00,4.496183e+00,8.085767e+00,2.216101e+00,2.210361e+00,5.466905e+00,3.037131e+00,6.294158e+00,6.517746e+00


In [12]:
lr = LinearRegression()
feature_candidates = list(features_engineered.columns)
features_used = []
R2 = []

while (len(feature_candidates) > 0):
    print('candidates left:',len(feature_candidates),end='')
    (feature_selected,R2_max) = SelectFeature(feature_candidates,features_used,targets,features_engineered)
    features_used.append(feature_selected)
    feature_candidates.remove(feature_selected)
    R2.append(R2_max)
    print('  selected:',feature_selected)
    
results = pd.DataFrame()
results['features'] = features_used
results['test R-squared'] = R2

print()
print(results)
ix = results['test R-squared'].idxmax()
print()
print('use features 0 to',ix,end='')
R2_max = results['test R-squared'].max()
print('    maximum test R-squared =',R2_max.round(3))

candidates left: 44

C:\Users\micha\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


  selected: Superplasticizer (component 5)(kg in a m^3 mixture) Age (day)
candidates left: 43  selected: Cement (component 1)(kg in a m^3 mixture)
candidates left: 42  selected: Blast Furnace Slag (component 2)(kg in a m^3 mixture) Coarse Aggregate  (component 6)(kg in a m^3 mixture)
candidates left: 41  selected: Fly Ash (component 3)(kg in a m^3 mixture) Age (day)
candidates left: 40  selected: Coarse Aggregate  (component 6)(kg in a m^3 mixture) Age (day)
candidates left: 39  selected: Water  (component 4)(kg in a m^3 mixture)
candidates left: 38  selected: Age (day)^2
candidates left: 37  selected: Cement (component 1)(kg in a m^3 mixture) Fly Ash (component 3)(kg in a m^3 mixture)
candidates left: 36  selected: Blast Furnace Slag (component 2)(kg in a m^3 mixture) Age (day)
candidates left: 35  selected: Blast Furnace Slag (component 2)(kg in a m^3 mixture) Superplasticizer (component 5)(kg in a m^3 mixture)
candidates left: 34  selected: Superplasticizer (component 5)(kg in a m^3

#### The engineered features increase the R-squared: 0.627 (from previous loop) to 0.796

In [13]:
lr.fit(features_engineered[features_used],targets)
coef = pd.Series(lr.coef_,index=features_engineered.columns)
coef.sort_values(ascending=False)

Fly Ash (component 3)(kg in a m^3 mixture) Superplasticizer (component 5)(kg in a m^3 mixture)                273.938489
Coarse Aggregate  (component 6)(kg in a m^3 mixture)                                                          260.490521
Blast Furnace Slag (component 2)(kg in a m^3 mixture)                                                         252.501233
Coarse Aggregate  (component 6)(kg in a m^3 mixture)^2                                                        223.032798
Water  (component 4)(kg in a m^3 mixture) Fine Aggregate (component 7)(kg in a m^3 mixture)                   136.899941
Fine Aggregate (component 7)(kg in a m^3 mixture) Age (day)                                                   116.770381
Blast Furnace Slag (component 2)(kg in a m^3 mixture) Coarse Aggregate  (component 6)(kg in a m^3 mixture)     35.257012
Blast Furnace Slag (component 2)(kg in a m^3 mixture)^2                                                        15.592522
Blast Furnace Slag (component 2)